In [ ]:
import pickle
import time
import datetime
import os.path
import matplotlib.pyplot as plt
import numpy as np
import random
import math

f = open('history.dat', 'rb')
history_arr = pickle.load(f)


In [ ]:
def make_chart_arr(arr):
    
    tm = []
    val = []
    
    for v in arr:
        open =  float(v[1])
        high =  float(v[2])
        low =   float(v[3])
        close = float(v[4])
        
        start_time = v[0]
        end_time = v[6]
        
        up_rotated = True
        if(len(val)>=4):
            up_rotated = open <= val[-4]
        
        tm.append(start_time)
        val.append(open)

        line_points = 5
        
        if up_rotated:
            for j in range(1,line_points):
                val.append(open+(high-open)*j/line_points)
                
            val.append(high)
            
            for j in range(1,line_points):
                val.append(high+(low-high)*j/line_points)
                
            val.append(low)
        
            for j in range(1,line_points):
                val.append(low+(close-low)*j/line_points)
        else:
            for j in range(1,line_points):
                val.append(open+(low-open)*j/line_points)
                
            val.append(low)
            
            for j in range(1,line_points):
                val.append(low+(high-low)*j/line_points)
                
            val.append(high)
        
            for j in range(1,line_points):
                val.append(high+(close-high)*j/line_points)
        
        for j in range(1,line_points*3):
            tm.append(int(start_time+(end_time-start_time)*j/(line_points*3)) )

        tm.append(end_time)
        val.append(close)
    
    return [tm,val]






In [ ]:
def findDateIndex(date):
    unix_time = int(datetime.datetime.timestamp(date)*1000)
    index = 0
    for v in history_arr:
        if v[0]>=unix_time:
            break
        index += 1;
    return index
    
#from_index = findDateIndex(datetime.datetime(2022, 6, 13, 0, 0,0))
#from_index = findDateIndex(datetime.datetime(2022, 6, 13, 0, 0,0))
from_index = findDateIndex(datetime.datetime(2021, 7, 17, 0, 0,0))
to_index = findDateIndex(datetime.datetime(2022, 7, 17, 0, 0,0))

#x,y = make_chart_arr(history_arr)
x,y = make_chart_arr(history_arr[from_index:to_index])
#x,y = make_chart_arr(history_arr[from_index:])

min_price = min(y)
max_price = max(y)

date_time = datetime.datetime.fromtimestamp(x[0]/1000)
print("from=", date_time.strftime('%Y-%m-%d %H:%M:%S'))

date_time = datetime.datetime.fromtimestamp(x[-1]/1000)
print("to=", date_time.strftime('%Y-%m-%d %H:%M:%S'))

print("min_price=", min_price)
print("max_price=", max_price)
print("start_price=", y[0])
print("stop_price=", y[-1])


In [ ]:
day_x=np.arange(0,len(x),dtype=float)
IDX_IN_DAY = 24*60*16
day_x /= IDX_IN_DAY

print(day_x[-1])

In [ ]:

plt.plot(day_x,y)
plt.show()


In [ ]:
import warnings
warnings.simplefilter("ignore", np.ComplexWarning)

#reduce_value=int(len(y)/28)
reduce_value=16

ry = np.array(y).reshape(-1,reduce_value ).mean(axis=1)
rday_x = np.array(day_x).reshape(-1, reduce_value).mean(axis=1)

fftY = np.fft.fft(ry)

def periodToTick(y_period):
    factor = len(day_x)/y_period
    return math.log2(factor)+1



plt.figure(figsize = (12, 6))
plt.subplot(121)

plt.vlines(periodToTick(IDX_IN_DAY*31),0,5000,color='red')
plt.vlines(periodToTick(IDX_IN_DAY*7),0,5000,color='red')
plt.vlines(periodToTick(IDX_IN_DAY),0,5000,color='red')
plt.vlines(periodToTick(60*16),0,5000,color='red')
plt.vlines(periodToTick(15*16),0,5000,color='red')
plt.vlines(periodToTick(16),0,5000,color='red')
plt.stem(np.abs(fftY)/(len(ry)/2),markerfmt=" ", basefmt="-b")
plt.xlabel('Freq (Hz)')
plt.ylabel('Price')
plt.xlim(0, 50)
plt.ylim(0, 5000)

plt.subplot(122)
plt.plot(day_x,y)
plt.plot(rday_x, np.fft.ifft(fftY), 'r')
plt.xlabel('Days (s)')
plt.ylabel('Price')

plt.tight_layout()
plt.show();


In [ ]:
# sampling rate
sr = 2000
# sampling interval
ts = 1.0/sr
t = np.arange(0,1.5,ts)

freq = 1.
#x = np.sin(2*np.pi*freq*t)

freq = 4
#x += np.sin(2*np.pi*freq*t)

freq = 100   
#x += 0.5* np.sin(2*np.pi*freq*t)
x = 0.5* np.sin(2*np.pi*freq*t)

#x = x[0:-100]
#t = t[0:-100]

plt.figure(figsize = (8, 6))
plt.plot(t, x, 'r')
plt.ylabel('Amplitude')

plt.show()

In [ ]:
from numpy.fft import fft, ifft

X = fft(x)
N = len(X)
n = np.arange(N)
T = N/sr
freq = n/T 

print(X)

plt.figure(figsize = (12, 6))
plt.subplot(121)

plt.stem(freq, np.abs(X), 'b', \
         markerfmt=" ", basefmt="-b")
plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')
plt.xlim(90, 110)

plt.subplot(122)

iF = x.copy()
#iF -=  ( X[1].real*np.cos(2*np.pi*1*t)-X[1].imag*np.sin(2*np.pi*1*t) ) / (len(X)/2)

plt.plot(t,iF , 'r')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.tight_layout()
plt.show()